In [156]:
import pandas as pd
import numpy as np

# Hybrid Recommender System

## Exercise 1
In this exercise, we are going to try out different methods, that can be used to combine rankings from multiple models.

Below is given a dataframe with the scores of how likely a user will like 5 different items estimated with 2 different models (rating predictions from a collaborative filtering model and cosine similarities from a content-based model).

In [157]:
d = {'item_id': ['I1', 'I2', 'I3', 'I4', 'I5'],
     'model1_score': [1.2, 2.8, 3.0, 4.5, 5.0],
     'model2_score': [0.8, 0.5, 0.2, 0.9, 0.4]}
df = pd.DataFrame(data=d)

### 1.1
Rank the 5 items according to the scores from model 1 and 2 respectively (in both cases higher score is better).

In [158]:
df[["m1_rank", "m2_rank"]] = df[["model1_score", "model2_score"]].rank(0, ascending=False)
df

,item_id,model1_score,model2_score,m1_rank,m2_rank
0,I1,1.2,0.8,5.0,2.0
1,I2,2.8,0.5,4.0,3.0
2,I3,3.0,0.2,3.0,5.0
3,I4,4.5,0.9,2.0,1.0
4,I5,5.0,0.4,1.0,4.0


### 1.2
Normalize the scores from the 2 models (by subtracting the mean and dividing with the standard deviation) and compute a combined rank using the Weighted Sum method with $\alpha=\beta=1$.

In [159]:
normalize = lambda x: (x - x.mean())/x.std()

alpha, beta = 1, 1
df[["model1_score", "model2_score"]] = df[["model1_score", "model2_score"]].apply(normalize)
df["weighted_sum"] =  alpha * df["model1_score"] + beta * df["model2_score"]
df

,item_id,model1_score,model2_score,m1_rank,m2_rank,weighted_sum
0,I1,-1.393819,0.833052,5.0,2.0,-0.560767
1,I2,-0.331862,-0.208263,4.0,3.0,-0.540125
2,I3,-0.199117,-1.249578,3.0,5.0,-1.448695
3,I4,0.796468,1.180157,2.0,1.0,1.976625
4,I5,1.128330,-0.555368,1.0,4.0,0.572962


### 1.3
Merge the ranking from the 2 models using the Borda Fuse method.

In [160]:
df["borda_points"] = 2*len(norm) - df["m1_rank"] - df["m2_rank"] 
df

,item_id,model1_score,model2_score,m1_rank,m2_rank,weighted_sum,borda_points
0,I1,-1.393819,0.833052,5.0,2.0,-0.560767,3.0
1,I2,-0.331862,-0.208263,4.0,3.0,-0.540125,3.0
2,I3,-0.199117,-1.249578,3.0,5.0,-1.448695,2.0
3,I4,0.796468,1.180157,2.0,1.0,1.976625,7.0
4,I5,1.128330,-0.555368,1.0,4.0,0.572962,5.0


### 1.4
Merge the ranking from the 2 models using the Reciprocal Rank Fusion method with $k=0$.

In [161]:
k = 0

df["rrf"] =  1 / (k + df["m1_rank"])
df["rrf"] += 1 / (k + df["m2_rank"])
df
# 1.333333
# 1	I2	-0.331862	-0.208263	-0.540125	3	0.700000
# 2	I3	-0.199117	-1.249578	-1.448695	5	0.833333
# 3	I4	0.796468	1.180157	1.976625	5	1.250000
# 4	I5	1.128330	-0.555368	0.572962	1	0.450000

,item_id,model1_score,model2_score,m1_rank,m2_rank,weighted_sum,borda_points,rrf
0,I1,-1.393819,0.833052,5.0,2.0,-0.560767,3.0,0.700000
1,I2,-0.331862,-0.208263,4.0,3.0,-0.540125,3.0,0.583333
2,I3,-0.199117,-1.249578,3.0,5.0,-1.448695,2.0,0.533333
3,I4,0.796468,1.180157,2.0,1.0,1.976625,7.0,1.500000
4,I5,1.128330,-0.555368,1.0,4.0,0.572962,5.0,1.250000


## Exercise 2
In this exercise, we are going to predict the rating of a single user-item pair using a hybrid method, where we use the user profiles from a content-based method as input to a (neighborhood-based) collaborative filtering method.

Below is given a dataframe with content-based user profiles of the user with 'reviewerID'='A25C2M3QF9G7OQ' and all users that have rated the item with 'asin'='B00EYZY6LQ'.

In [162]:
up = {'action': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.2340145069033483, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.19501208908612358, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'advanced': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.04290793709094304}, 'aerosol': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.3651061551295451, 'A1R1BFJCMWX0Y3': 0.2738296163471588, 'A1UQBFCERIP7VJ': 0.20283675284974728, 'A22CW0ZHY3NJH8': 0.18255307756477254, 'A2LW5AL0KQ9P1M': 0.24340410341969673, 'A2PD27UKAD3Q00': 0.6085102585492418, 'A2WW57XX2UVLM6': 0.48680820683939346, 'A2ZY49IDE6TY5I': 0.29208492410363607, 'A39WWMBA0299ZF': 0.6085102585492418, 'A3M6TSEV71537G': 0.6085102585492418, 'A3S3R88HA0HZG3': 0.24340410341969673, 'A914TQVHI872U': 0.45638269391193137, 'AOEUN9718KVRD': 0.18255307756477254}, 'amp': {'A25C2M3QF9G7OQ': 0.16623276178709817, 'A1F7YU6O5RU432': 0.13837240135343984, 'A1R1BFJCMWX0Y3': 0.12467457134032363, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.11531033446119987, 'A2LW5AL0KQ9P1M': 0.21325040495887393, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.22416931040887636, 'A2ZY49IDE6TY5I': 0.16623276178709817, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.18680775867406363, 'A914TQVHI872U': 0.20779095223387273, 'AOEUN9718KVRD': 0.0782710244974592}, 'artisanal': {'A25C2M3QF9G7OQ': 0.28113176760896397, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.21084882570672298, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.23427647300746998, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.28113176760896397, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.35141470951120496, 'AOEUN9718KVRD': 0.0}, 'bamboo': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.3067455423637883, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.2726627043233674, 'A2LW5AL0KQ9P1M': 0.20449702824252555, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'bar': {'A25C2M3QF9G7OQ': 0.22702809062920143, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.17027106797190109, 'A1UQBFCERIP7VJ': 0.17001726730995562, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.18919007552433453, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.22702809062920143, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.2837851132865018, 'AOEUN9718KVRD': 0.0}, 'bath': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.24771250643126969, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.22018889460557303, 'A2LW5AL0KQ9P1M': 0.16514167095417978, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.10689662526336474}, 'blossoms': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.6235483649721686, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.6235483649721686, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.12470967299443374}, 'blown': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.3651061551295451, 'A1R1BFJCMWX0Y3': 0.2738296163471588, 'A1UQBFCERIP7VJ': 0.20283675284974728, 'A22CW0ZHY3NJH8': 0.18255307756477254, 'A2LW5AL0KQ9P1M': 0.24340410341969673, 'A2PD27UKAD3Q00': 0.6085102585492418, 'A2WW57XX2UVLM6': 0.48680820683939346, 'A2ZY49IDE6TY5I': 0.29208492410363607, 'A39WWMBA0299ZF': 0.6085102585492418, 'A3M6TSEV71537G': 0.6085102585492418, 'A3S3R88HA0HZG3': 0.24340410341969673, 'A914TQVHI872U': 0.45638269391193137, 'AOEUN9718KVRD': 0.18255307756477254}, 'body': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.1393671328806919, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'booties': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.07729470842199393, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.5797103131649545, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.2318841252659818, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.28985515658247724, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'butter': {'A25C2M3QF9G7OQ': 0.2571905091496767, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.19289288186225756, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.21432542429139725, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.2571905091496767, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.3214881364370959, 'AOEUN9718KVRD': 0.0}, 'cavity': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.20710205313498298, 'A22CW0ZHY3NJH8': 0.2485224637619796, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'clean': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.21408576729257414, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.19260536184089203, 'A22CW0ZHY3NJH8': 0.17840480607714512, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'cleanser': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.2340145069033483, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.19501208908612358, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'clear': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.21408576729257414, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.17840480607714512, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'colgate': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.3623148574875192, 'A22CW0ZHY3NJH8': 0.43477782898502304, 'A2LW5AL0KQ9P1M': 0.30159887421266895, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'cond': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.3651061551295451, 'A1R1BFJCMWX0Y3': 0.2738296163471588, 'A1UQBFCERIP7VJ': 0.20283675284974728, 'A22CW0ZHY3NJH8': 0.18255307756477254, 'A2LW5AL0KQ9P1M': 0.24340410341969673, 'A2PD27UKAD3Q00': 0.6085102585492418, 'A2WW57XX2UVLM6': 0.48680820683939346, 'A2ZY49IDE6TY5I': 0.29208492410363607, 'A39WWMBA0299ZF': 0.6085102585492418, 'A3M6TSEV71537G': 0.6085102585492418, 'A3S3R88HA0HZG3': 0.24340410341969673, 'A914TQVHI872U': 0.45638269391193137, 'AOEUN9718KVRD': 0.18255307756477254}, 'conditioner': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.16668545738397234, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.40004509772153357, 'A3S3R88HA0HZG3': 0.2500281860759585, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.036079809216672316}, 'cool': {'A25C2M3QF9G7OQ': 0.1937976759895667, 'A1F7YU6O5RU432': 0.3875953519791334, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.21533075109951857, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.2422470949869584}, 'count': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.5319887772144556, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.3904792383460515, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.2659943886072278, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.7093183696192741, 'A3S3R88HA0HZG3': 0.35465918480963704, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'couture': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.4026824351903138, 'A1R1BFJCMWX0Y3': 0.37751478299091923, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.5033530439878923, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.4194608699899102, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'cpc': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.14062234501142495, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.42186703503427486, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.14062234501142495}, 'cream': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.18001736190555268, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.15001446825462725, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'crest': {'A25C2M3QF9G7OQ': 0.15650140403017584, 'A1F7YU6O5RU432': 0.8449915852748073, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.5079120605472304, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.2659943886072278, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.34067892176615056, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.7093183696192741, 'A3S3R88HA0HZG3': 0.35465918480963704, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.3091863956264367}, 'daily': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.2340145069033483, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.19501208908612358, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'de': {'A25C2M3QF9G7OQ': 0.20716061345241324, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.15537046008930994, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.1726338445436777, 'A2PD27UKAD3Q00': 0.45948084381754145, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.20716061345241324, 'A39WWMBA0299ZF': 0.45948084381754145, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.25895076681551654, 'AOEUN9718KVRD': 0.18943815312402426}, 'deep': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.21408576729257414, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.19260536184089203, 'A22CW0ZHY3NJH8': 0.17840480607714512, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'dental': {'A25C2M3QF9G7OQ': 0.4026126133460143, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.2684084088973429, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'dove': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.2105345411240737, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'dried': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.6235483649721686, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.6235483649721686, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.12470967299443374}, 'enamel': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.35298768666274555, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'enriched': {'A25C2M3QF9G7OQ': 0.28113176760896397, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.21084882570672298, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.23427647300746998, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.28113176760896397, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.35141470951120496, 'AOEUN9718KVRD': 0.0}, 'essie': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.3683898885318512, 'A1R1BFJCMWX0Y3': 0.34536552049861047, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.460487360664814, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.38373946722067837, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'eyelash': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.04690213584752027}, 'face': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.18001736190555268, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.16195522741162832, 'A22CW0ZHY3NJH8': 0.15001446825462725, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'facial': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.21408576729257414, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.17840480607714512, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'feet': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.07729470842199393, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.5797103131649545, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.2318841252659818, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.28985515658247724, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'fekkai': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.3651061551295451, 'A1R1BFJCMWX0Y3': 0.2738296163471588, 'A1UQBFCERIP7VJ': 0.20283675284974728, 'A22CW0ZHY3NJH8': 0.18255307756477254, 'A2LW5AL0KQ9P1M': 0.24340410341969673, 'A2PD27UKAD3Q00': 0.6085102585492418, 'A2WW57XX2UVLM6': 0.48680820683939346, 'A2ZY49IDE6TY5I': 0.29208492410363607, 'A39WWMBA0299ZF': 0.6085102585492418, 'A3M6TSEV71537G': 0.6085102585492418, 'A3S3R88HA0HZG3': 0.24340410341969673, 'A914TQVHI872U': 0.45638269391193137, 'AOEUN9718KVRD': 0.18255307756477254}, 'fl': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.09967805382437057, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.29903416147311174, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.09967805382437057}, 'floss': {'A25C2M3QF9G7OQ': 1.6372306243539398, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.4586657099458307, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.949232059435194, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'fluid': {'A25C2M3QF9G7OQ': 0.1325764088886735, 'A1F7YU6O5RU432': 0.265152817777347, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.2955397411453269, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.08645017771634665, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.25935053314903994, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.3557582883789885, 'A3S3R88HA0HZG3': 0.4384743745277344, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.16572051111084188}, 'fluoride': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.21694688843327928, 'A22CW0ZHY3NJH8': 0.26033626611993516, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'foam': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.3651061551295451, 'A1R1BFJCMWX0Y3': 0.2738296163471588, 'A1UQBFCERIP7VJ': 0.20283675284974728, 'A22CW0ZHY3NJH8': 0.18255307756477254, 'A2LW5AL0KQ9P1M': 0.24340410341969673, 'A2PD27UKAD3Q00': 0.6085102585492418, 'A2WW57XX2UVLM6': 0.48680820683939346, 'A2ZY49IDE6TY5I': 0.29208492410363607, 'A39WWMBA0299ZF': 0.6085102585492418, 'A3M6TSEV71537G': 0.6085102585492418, 'A3S3R88HA0HZG3': 0.24340410341969673, 'A914TQVHI872U': 0.45638269391193137, 'AOEUN9718KVRD': 0.18255307756477254}, 'fragr': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.6235483649721686, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.6235483649721686, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.12470967299443374}, 'french': {'A25C2M3QF9G7OQ': 0.22702809062920143, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.17027106797190109, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.18919007552433453, 'A2PD27UKAD3Q00': 0.503546774235407, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.22702809062920143, 'A39WWMBA0299ZF': 0.503546774235407, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.2837851132865018, 'AOEUN9718KVRD': 0.20760598011044615}, 'full': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.3651061551295451, 'A1R1BFJCMWX0Y3': 0.2738296163471588, 'A1UQBFCERIP7VJ': 0.20283675284974728, 'A22CW0ZHY3NJH8': 0.18255307756477254, 'A2LW5AL0KQ9P1M': 0.24340410341969673, 'A2PD27UKAD3Q00': 0.6085102585492418, 'A2WW57XX2UVLM6': 0.48680820683939346, 'A2ZY49IDE6TY5I': 0.29208492410363607, 'A39WWMBA0299ZF': 0.6085102585492418, 'A3M6TSEV71537G': 0.6085102585492418, 'A3S3R88HA0HZG3': 0.24340410341969673, 'A914TQVHI872U': 0.45638269391193137, 'AOEUN9718KVRD': 0.18255307756477254}, 'gel': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.29672897164922596, 'A1R1BFJCMWX0Y3': 0.27818341092114934, 'A1UQBFCERIP7VJ': 0.159864005682001, 'A22CW0ZHY3NJH8': 0.19183680681840118, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.3709112145615325, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.30909267880127705, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.09754198436051599}, 'gentle': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.2340145069033483, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.19501208908612358, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'gingivitis': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.2730008583350251, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'glamour': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.12637122751138535, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.37911368253415606, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.3159280687784634, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'glide': {'A25C2M3QF9G7OQ': 0.4026126133460143, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.2684084088973429, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'grams': {'A25C2M3QF9G7OQ': 0.28113176760896397, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.21084882570672298, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.23427647300746998, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.28113176760896397, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.35141470951120496, 'AOEUN9718KVRD': 0.0}, 'guy': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.12637122751138535, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.37911368253415606, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.3159280687784634, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'health': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.3229271584228583, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'hellip': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.2105345411240737, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'hydrated': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.07729470842199393, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.5797103131649545, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.2318841252659818, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.28985515658247724, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'jute': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.3067455423637883, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.2726627043233674, 'A2LW5AL0KQ9P1M': 0.20449702824252555, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'keep': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.07729470842199393, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.5797103131649545, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.2318841252659818, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.28985515658247724, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'kids': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.20710205313498298, 'A22CW0ZHY3NJH8': 0.2485224637619796, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'kit': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.2730008583350251, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'lather': {'A25C2M3QF9G7OQ': 0.28113176760896397, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.21084882570672298, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.23427647300746998, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.28113176760896397, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.35141470951120496, 'AOEUN9718KVRD': 0.0}, 'lavender': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.503546774235407, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.503546774235407, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.20760598011044615}, 'lectric': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 1.4339268936889131, 'AOEUN9718KVRD': 0.0}, 'life': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.14062234501142495, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.42186703503427486, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.14062234501142495}, 'liquid': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.19847163189399064, 'A22CW0ZHY3NJH8': 0.23816595827278875, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'maison': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.5704468151379668, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.5704468151379668, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.2351880226765889}, 'maximum': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.20710205313498298, 'A22CW0ZHY3NJH8': 0.2485224637619796, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'may': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.18513829461003276, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.4443319070640786, 'A3S3R88HA0HZG3': 0.27770744191504915, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'mint': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.1286469075299077, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.3859407225897231, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.1286469075299077}, 'mitt': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.3067455423637883, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.2726627043233674, 'A2LW5AL0KQ9P1M': 0.20449702824252555, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'ml': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.04690213584752027}, 'moistur': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.07729470842199393, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.5797103131649545, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.2318841252659818, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.28985515658247724, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'moisturizing': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.07729470842199393, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.5797103131649545, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.2318841252659818, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.28985515658247724, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'mouthwash': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.35298768666274555, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'nail': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.34405893946847527, 'A1R1BFJCMWX0Y3': 0.3225552557516955, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.43007367433559407, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.3583947286129951, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'natural': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.24771250643126969, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.22018889460557303, 'A2LW5AL0KQ9P1M': 0.16514167095417978, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'original': {'A25C2M3QF9G7OQ': 0.3683260127349681, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.24555067515664541, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.7357830047498732, 'AOEUN9718KVRD': 0.0}, 'ounc': {'A25C2M3QF9G7OQ': 0.10668941780431496, 'A1F7YU6O5RU432': 0.414376927709797, 'A1R1BFJCMWX0Y3': 0.1507485690758753, 'A1UQBFCERIP7VJ': 0.23020940428322056, 'A22CW0ZHY3NJH8': 0.10049904605058353, 'A2LW5AL0KQ9P1M': 0.20356856486534405, 'A2PD27UKAD3Q00': 0.3349968201686118, 'A2WW57XX2UVLM6': 0.47670696652858746, 'A2ZY49IDE6TY5I': 0.16079847368093367, 'A39WWMBA0299ZF': 0.3349968201686118, 'A3M6TSEV71537G': 0.3349968201686118, 'A3S3R88HA0HZG3': 0.3079233200621931, 'A914TQVHI872U': 0.6713669859399725, 'AOEUN9718KVRD': 0.23386081830597724}, 'ounce': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.4674184192296498, 'A22CW0ZHY3NJH8': 0.18313144219705482, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.3832080342331697, 'A3S3R88HA0HZG3': 0.23950502139573104, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'oz': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.2670376291752138, 'A1R1BFJCMWX0Y3': 0.12205022490659653, 'A1UQBFCERIP7VJ': 0.2497818810786591, 'A22CW0ZHY3NJH8': 0.2843228368666118, 'A2LW5AL0KQ9P1M': 0.1084890888058636, 'A2PD27UKAD3Q00': 0.27122272201465897, 'A2WW57XX2UVLM6': 0.4050110397742152, 'A2ZY49IDE6TY5I': 0.1301869065670363, 'A39WWMBA0299ZF': 0.27122272201465897, 'A3M6TSEV71537G': 0.27122272201465897, 'A3S3R88HA0HZG3': 0.1084890888058636, 'A914TQVHI872U': 0.5618953678852483, 'AOEUN9718KVRD': 0.16494946690220394}, 'pack': {'A25C2M3QF9G7OQ': 0.25845713408975884, 'A1F7YU6O5RU432': 0.15022559349795453, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.4073708146257283, 'A22CW0ZHY3NJH8': 0.6241545705204312, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'pantene': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.19823079097939317, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.4757538983505436, 'A3S3R88HA0HZG3': 0.2973461864690898, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'plumping': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.12637122751138535, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.37911368253415606, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.3159280687784634, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'polish': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.3683898885318512, 'A1R1BFJCMWX0Y3': 0.34536552049861047, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.460487360664814, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.38373946722067837, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'pre': {'A25C2M3QF9G7OQ': 0.24020391594364887, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.18015293695773665, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.20016992995304073, 'A2PD27UKAD3Q00': 0.5327706659423385, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.24020391594364887, 'A39WWMBA0299ZF': 0.5327706659423385, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.3002548949295611, 'AOEUN9718KVRD': 0.2196546218471971}, 'product': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.2166836282054536, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.5200407076930886, 'A3S3R88HA0HZG3': 0.32502544230818037, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'professional': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.24975203034025636, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'protection': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.1894651854706701, 'A22CW0ZHY3NJH8': 0.22735822256480412, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'provence': {'A25C2M3QF9G7OQ': 0.24020391594364887, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.18015293695773665, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.20016992995304073, 'A2PD27UKAD3Q00': 0.5327706659423385, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.24020391594364887, 'A39WWMBA0299ZF': 0.5327706659423385, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.3002548949295611, 'AOEUN9718KVRD': 0.2196546218471971}, 'pump': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.1894651854706701, 'A22CW0ZHY3NJH8': 0.22735822256480412, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'raspberri': {'A25C2M3QF9G7OQ': 0.28113176760896397, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.21084882570672298, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.23427647300746998, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.28113176760896397, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.35141470951120496, 'AOEUN9718KVRD': 0.0}, 'revitalash': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.1407064075425608}, 'rich': {'A25C2M3QF9G7OQ': 0.28113176760896397, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.21084882570672298, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.23427647300746998, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.28113176760896397, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.35141470951120496, 'AOEUN9718KVRD': 0.0}, 'rinse': {'A25C2M3QF9G7OQ': 0.1772938127330705, 'A1F7YU6O5RU432': 0.354587625466141, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.32564003278887493, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.3859407225897231, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.3502641734462458}, 'sensitive': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.2340145069033483, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.19501208908612358, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'shave': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 1.1549927558548947, 'AOEUN9718KVRD': 0.0}, 'shea': {'A25C2M3QF9G7OQ': 0.2571905091496767, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.19289288186225756, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.21432542429139725, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.2571905091496767, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.3214881364370959, 'AOEUN9718KVRD': 0.0}, 'shower': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.10689662526336474}, 'size': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.19823079097939317, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.4757538983505436, 'A3S3R88HA0HZG3': 0.2973461864690898, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'skin': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.18897852467423268, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.1574821038951939, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'smash': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.21694688843327928, 'A22CW0ZHY3NJH8': 0.26033626611993516, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'smooth': {'A25C2M3QF9G7OQ': 0.21626265748436166, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.16219699311327124, 'A1UQBFCERIP7VJ': 0.1676342787138302, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.18021888123696805, 'A2PD27UKAD3Q00': 0.44594637583087265, 'A2WW57XX2UVLM6': 0.32452428580914155, 'A2ZY49IDE6TY5I': 0.21626265748436166, 'A39WWMBA0299ZF': 0.17837855033234906, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.22297318791543633, 'A914TQVHI872U': 0.2703283218554521, 'AOEUN9718KVRD': 0.10817476193638052}, 'soap': {'A25C2M3QF9G7OQ': 0.24020391594364887, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.18015293695773665, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.20016992995304073, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.24020391594364887, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.3002548949295611, 'AOEUN9718KVRD': 0.0}, 'spa': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.2806230078191312, 'A1UQBFCERIP7VJ': 0.07071226984664171, 'A22CW0ZHY3NJH8': 0.24944267361700548, 'A2LW5AL0KQ9P1M': 0.18708200521275412, 'A2PD27UKAD3Q00': 0.5303420238498128, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.21213680953992511, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.2651710119249064, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'strawberry': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.21694688843327928, 'A22CW0ZHY3NJH8': 0.26033626611993516, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'strips': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.6587684584402448, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.3293842292201224, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.878357944586993, 'A3S3R88HA0HZG3': 0.4391789722934965, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'toni': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.12637122751138535, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.37911368253415606, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.3159280687784634, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'toothpaste': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.38793681736466074, 'A22CW0ZHY3NJH8': 0.4655241808375929, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'ultra': {'A25C2M3QF9G7OQ': 1.0163592700320787, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 1.0163592700320787, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'urban': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.2806230078191312, 'A1UQBFCERIP7VJ': 0.07071226984664171, 'A22CW0ZHY3NJH8': 0.24944267361700548, 'A2LW5AL0KQ9P1M': 0.18708200521275412, 'A2PD27UKAD3Q00': 0.5303420238498128, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.21213680953992511, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.2651710119249064, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'us': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.3651061551295451, 'A1R1BFJCMWX0Y3': 0.2738296163471588, 'A1UQBFCERIP7VJ': 0.20283675284974728, 'A22CW0ZHY3NJH8': 0.18255307756477254, 'A2LW5AL0KQ9P1M': 0.24340410341969673, 'A2PD27UKAD3Q00': 0.6085102585492418, 'A2WW57XX2UVLM6': 0.48680820683939346, 'A2ZY49IDE6TY5I': 0.29208492410363607, 'A39WWMBA0299ZF': 0.6085102585492418, 'A3M6TSEV71537G': 0.6085102585492418, 'A3S3R88HA0HZG3': 0.24340410341969673, 'A914TQVHI872U': 0.45638269391193137, 'AOEUN9718KVRD': 0.18255307756477254}, 'vary': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.18513829461003276, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.4443319070640786, 'A3S3R88HA0HZG3': 0.27770744191504915, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'volume': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.19823079097939317, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.11560941910602678, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.3468282573180803, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.4757538983505436, 'A3S3R88HA0HZG3': 0.5863697342341566, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'wash': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.1724408067404613, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.14370067228371775, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'water': {'A25C2M3QF9G7OQ': 1.0163592700320787, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 1.0163592700320787, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'waterpik': {'A25C2M3QF9G7OQ': 1.0163592700320787, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 1.0163592700320787, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'whip': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.12637122751138535, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.37911368253415606, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.3159280687784634, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.0}, 'williams': {'A25C2M3QF9G7OQ': 0.0, 'A1F7YU6O5RU432': 0.0, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.0, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.7631324584711068, 'AOEUN9718KVRD': 0.0}, 'wintergreen': {'A25C2M3QF9G7OQ': 0.1937976759895667, 'A1F7YU6O5RU432': 0.3875953519791334, 'A1R1BFJCMWX0Y3': 0.0, 'A1UQBFCERIP7VJ': 0.21533075109951857, 'A22CW0ZHY3NJH8': 0.0, 'A2LW5AL0KQ9P1M': 0.0, 'A2PD27UKAD3Q00': 0.0, 'A2WW57XX2UVLM6': 0.0, 'A2ZY49IDE6TY5I': 0.0, 'A39WWMBA0299ZF': 0.0, 'A3M6TSEV71537G': 0.0, 'A3S3R88HA0HZG3': 0.0, 'A914TQVHI872U': 0.0, 'AOEUN9718KVRD': 0.2422470949869584}}
user_profiles = pd.DataFrame(up)

### 2.1
Compute the cosine similarities between user 'A25C2M3QF9G7OQ' and the other users based on their user profiles.<br>
What are the similarities and what are the ratings given by these users on item 'B00EYZY6LQ'?

In [166]:
df = pd.read_pickle("train.pkl")
df = df[["overall", "reviewerID", "asin"]]

df[df["asin"] == "B00EYZY6LQ"]["reviewerID"]

# df[df["asin"] == "B00EYZY6LQ"]["reviewerID"].isin(user_profiles.index)

4774    A1F7YU6O5RU432
4777    A1R1BFJCMWX0Y3
4770    A1UQBFCERIP7VJ
4772    A22CW0ZHY3NJH8
4767    A2LW5AL0KQ9P1M
4776    A2PD27UKAD3Q00
4779    A2WW57XX2UVLM6
4769    A2ZY49IDE6TY5I
4778    A39WWMBA0299ZF
4771    A3M6TSEV71537G
4775    A3S3R88HA0HZG3
4768     A914TQVHI872U
4773     AOEUN9718KVRD
Name: reviewerID, dtype: object

In [175]:
user_profiles#.reset_index().rename(columns={"index":"reviewerID"})



,action,advanced,aerosol,amp,artisanal,bamboo,bar,bath,blossoms,blown,...,urban,us,vary,volume,wash,water,waterpik,whip,williams,wintergreen
A25C2M3QF9G7OQ,0.000000,0.000000,0.000000,0.166233,0.281132,0.000000,0.227028,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.016359,1.016359,0.000000,0.000000,0.193798
A1F7YU6O5RU432,0.234015,0.000000,0.365106,0.138372,0.000000,0.000000,0.000000,0.000000,0.000000,0.365106,...,0.000000,0.365106,0.000000,0.000000,0.172441,0.000000,0.000000,0.000000,0.000000,0.387595
A1R1BFJCMWX0Y3,0.000000,0.000000,0.273830,0.124675,0.210849,0.306746,0.170271,0.247713,0.000000,0.273830,...,0.280623,0.273830,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A1UQBFCERIP7VJ,0.000000,0.000000,0.202837,0.000000,0.000000,0.000000,0.170017,0.000000,0.000000,0.202837,...,0.070712,0.202837,0.185138,0.198231,0.000000,0.000000,0.000000,0.000000,0.000000,0.215331
A22CW0ZHY3NJH8,0.195012,0.000000,0.182553,0.115310,0.000000,0.272663,0.000000,0.220189,0.000000,0.182553,...,0.249443,0.182553,0.000000,0.000000,0.143701,0.000000,0.000000,0.000000,0.000000,0.000000
A2LW5AL0KQ9P1M,0.000000,0.000000,0.243404,0.213250,0.234276,0.204497,0.189190,0.165142,0.000000,0.243404,...,0.187082,0.243404,0.000000,0.115609,0.000000,0.000000,0.000000,0.126371,0.000000,0.000000
A2PD27UKAD3Q00,0.000000,0.000000,0.608510,0.000000,0.000000,0.000000,0.000000,0.000000,0.623548,0.608510,...,0.530342,0.608510,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A2WW57XX2UVLM6,0.000000,0.000000,0.486808,0.224169,0.000000,0.000000,0.000000,0.000000,0.000000,0.486808,...,0.000000,0.486808,0.000000,0.346828,0.000000,0.000000,0.000000,0.379114,0.000000,0.000000
A2ZY49IDE6TY5I,0.000000,0.000000,0.292085,0.166233,0.281132,0.000000,0.227028,0.000000,0.000000,0.292085,...,0.000000,0.292085,0.000000,0.000000,0.000000,1.016359,1.016359,0.000000,0.000000,0.000000
A39WWMBA0299ZF,0.000000,0.000000,0.608510,0.000000,0.000000,0.000000,0.000000,0.000000,0.623548,0.608510,...,0.212137,0.608510,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [215]:
cosine_sim = lambda X, Y: (X.T @ Y)/(np.linalg.norm(X)*np.linalg.norm(Y))

arr = user_profiles.loc["A25C2M3QF9G7OQ"]
up = user_profiles.copy().drop(index="A25C2M3QF9G7OQ")
up["sim"] = up.apply(lambda x: cosine_sim(arr, x), axis=1)#cosine_similarity(arr, x))
up["sim"]

A1F7YU6O5RU432    0.085150
A1R1BFJCMWX0Y3    0.186912
A1UQBFCERIP7VJ    0.097791
A22CW0ZHY3NJH8    0.263018
A2LW5AL0KQ9P1M    0.225416
A2PD27UKAD3Q00    0.075767
A2WW57XX2UVLM6    0.057745
A2ZY49IDE6TY5I    0.805320
A39WWMBA0299ZF    0.080646
A3M6TSEV71537G    0.028200
A3S3R88HA0HZG3    0.041409
A914TQVHI872U     0.202305
AOEUN9718KVRD     0.164220
Name: sim, dtype: float64

### 2.2
Predict the rating for user 'A25C2M3QF9G7OQ' on item 'B00EYZY6LQ' based on the ratings from the $3$ most similar users, using a weighted (by similarity) average.<br>
What is the prediction?

In [279]:
users = up.nlargest(3, "sim")["sim"].reset_index().rename(columns={"index":"reviewerID"})
merge = users.merge(df, on="reviewerID", how="left")

def weighted_avg(data, weights):
    return data.T @ (weights/weights.sum())

weighted_avg(merge["overall"], merge["sim"])

4.328007473081277